# Welcome!

Jupyter repo for this tutorial: https://github.com/reuven/2024-pycon-decorators/

# Agenda

1. What are decorators?
2. Writing your first decorator
3. Outer function storage
4. Inputs and outputs
5. Decorators that take arguments
6. Nested decorators
7. Decorating classes
8. Writing decorators as classes

# What are decorators?

Let's assume that we have two basic functions.

In [2]:
def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(a())
print(b())

A!

B!



In [3]:
lines = '-' * 60 + '\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



# Don't Repeat Yourself (DRY) rule of programming

Right now, we have a technique that will require us to include `lines` in every function we write!

How can I include `lines` without having to include that variable explicitly in every function?

In [4]:
# option 1: Write a new function that takes a function as an argument

lines = '-' * 40 + '\n'

def with_lines(func):
    return f'{lines}{func()}{lines}'

def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(with_lines(a))
print(with_lines(b))

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



In [5]:
# option 2: turn with_lines into a function that returns a function
# that is: We're going to create a closure!

lines = '-' * 40 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
with_lines_a = with_lines(a)

def b():
    return f'B!\n'

with_lines_b = with_lines(b)

print(with_lines_a())
print(with_lines_b())

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



In [7]:
# option 3: don't assign to with_lines_a and with_lines_b
# rather, assign to a and b

lines = '-' * 40 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
a = with_lines(a)

def b():
    return f'B!\n'
b = with_lines(b)

print(a())
print(b())

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



In [8]:
x = 5
x = 7
print(x)

7


In [9]:
# option 4: Use Python decorator syntax

lines = '-' * 40 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

@with_lines    # this is 100% equivalent to what's in line 13 -- decorator syntax
def a():
    return f'A!\n'
# a = with_lines(a)

@with_lines
def b():
    return f'B!\n'
# b = with_lines(b)

print(a())
print(b())

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



# Who needs this? 

1. if I have many functions that need to do the same thing, I can extract that functionality into a decorator, and thus DRY up my code.
2. Timing my functions
3. Security of my functions
4. Setting values before/after (setup/teardown)
5. Logging
6. Checking/filtering inputs
7. Filtering outputs

In [11]:
# let's decorate another function!

@with_lines
def add(x, y):
    return f'{x + y}\n'

print(add(3, 5))

TypeError: with_lines.<locals>.wrapper() takes 0 positional arguments but 2 were given

In [13]:

def with_lines(func):
    def wrapper(*args):   # take any number of positional arguments, create a tuple, args
        return f'{lines}{func(*args)}{lines}'  # turn args into arguments to func()
    return wrapper

@with_lines
def add(x, y):
    return f'{x + y}\n'

print(add(3, 5))

----------------------------------------
8
----------------------------------------



# How to write a decorator

1. The outer function, the decorator, takes one argument, a function (`func`) -- this is the function that will be decorated. This outer function is invoked once, when we define the decorated function.
2. The inner function, typically called `wrapper`, takes `*args` (and maybe `**kwargs`). It in invoked every time the original function is invoked. It can do whatever it wants with the arguments and outputs
3. Ater defining the inner function, the outer function returns `wrapper`.
4. We can then decorate any function with `@deco_name`

In [14]:
def mysum(numbers):
    total = 0
    for one_number in numbers:
        total += one_number
    return total

mysum([10, 20, 30])


60

In [15]:
mysum(10, 20, 30)

TypeError: mysum() takes 1 positional argument but 3 were given

In [16]:
def mysum(*numbers):   # now the function takes any number of positional arguments
    total = 0
    for one_number in numbers:
        total += one_number
    return total

mysum(10, 20, 30)


60

# Exercise: Shouter

1. Write a decorator, `shouter`, that decorates functions that return strings.
2. Any such function's output will return all `IN CAPS` and with an exclamation point at the end.

Example:

```python
@shouter
def hello(name):
    return f'Hello, {name}'

print(hello('Reuven'))      # output HELLO, REUVEN!
```

Hint:
1. Use `str.upper`
2. Use `!`

In [22]:
def shouter(func):   # outer function / decorator function
    def wrapper(*args):   # inner function -- runs each time we invoke hello
        return f'{func(*args).upper()}!'
    return wrapper

@shouter
def hello(name):
    return f'Hello, {name}'

print(hello('Reuven'))

HELLO, REUVEN!


# Exercise: Timing of functions

Write a decorator, `timefunc`, that will not change the inputs or outputs of the decorated function. However, it will keep track of how long it took to run the function, and will write its results to a file, `timing.txt`.

Every time I run the decorated function, I'll get another line in `timing.txt`, with (a) the function name, (b) when it was started, and (c) how long it took.

Hints:
1. Normally, writing to a file with `w` will overwrite the file's contents. You should open a file with `a` (append), and then that won't happen.
2. You can get the function name from the `__name__` attribute.
3. You can get the current Unix time (seconds since 1 Jan 1970) with `time.time()`

```python
import random
import time

@timefunc
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@timefunc
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))    
print(slow_mul(4, 5))
```

In [25]:
import random
import time

def timefunc(func):
    def wrapper(*args):
        start_time = time.time()
        value = func(*args)
        end_time = time.time()

        with open('timing.txt', 'a') as f:
            f.write(f'{func.__name__}\t{start_time}\t{end_time-start_time}\n')
        return value
    return wrapper

@timefunc
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@timefunc
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))    
print(slow_mul(4, 5))

5
20


In [26]:
!cat timing.txt

slow_add	1715884373.5334568	1.71661376953125e-05
slow_mul	1715884373.534719	1.4066696166992188e-05
slow_add	1715884378.5905652	1.0001816749572754
slow_mul	1715884379.591807	2.0012950897216797


# Using the outer function

When we have an iinner function, it has access (as we've seen) to the outer function's parameters and local variables. However, we can go further than this, storing data in the outer function. That data persists across calls to our inner (wrapper) function.

In [36]:
# I want to know how many times a function has been invoked, and print that
# to the screen every time we call it.

def count_calls(func):
    counter = 0
    def wrapper(*args):
        value = func(*args)
        counter += 1    # counter = counter + 1
        print(f'{func.__name__}: {counter=}')
        return value
    return wrapper

@count_calls
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@count_calls
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))    
print(slow_mul(4, 5))
print(slow_add(2, 3))    
print(slow_mul(4, 5))
print(slow_add(2, 3))    
print(slow_mul(4, 5))

UnboundLocalError: cannot access local variable 'counter' where it is not associated with a value

In [6]:
# nonlocal to the rescule

import time
import random

def count_calls(func):
    counter = 0
    def wrapper(*args):
        nonlocal counter
        value = func(*args)
        counter += 1
        print(f'{func.__name__}: {counter=}')
        return value
    return wrapper

@count_calls
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@count_calls
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))    
print(slow_add(2, 3))    
print(slow_add(2, 3))    
print(slow_mul(4, 5))

slow_add: counter=1
5
slow_add: counter=2
5
slow_add: counter=3
5
slow_mul: counter=1
20


# Memoization

This is an old technique for caching. The idea is that every time we call a function, we want to check the arguments passed to the function:

- If the arguments haven't been seen before, we run the function and cache the result
- If the arguments have been seen before, then just grab the results from the cache and return them



In [11]:
def memoize(func):
    cache = {}
    def wrapper(*args):
        if args not in cache:
            print(f'\tActually calling {func.__name__} with {args}')
            cache[args] = func(*args)
        return cache[args]
    return wrapper

@memoize
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@memoize
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))    
print(slow_add(2, 3))    
print(slow_add(3, 4))    
print(slow_mul(2, 3))
print(slow_mul(2, 3))
print(slow_mul(1, 2))

	Actually calling slow_add with (2, 3)
5
5
	Actually calling slow_add with (3, 4)
7
	Actually calling slow_mul with (2, 3)
6
6
	Actually calling slow_mul with (1, 2)
2


# Exercise: `once_per_minute`

Write a decorator, `once_per_minute`, that ensures a function can only run once in any 60-second period. Every time the function starts, it checks to make sure that at least 60 seconds have passed. 

If 60 seconds haven't passed since the last run, we should raise an exception.

If they have passed, then reset the counter to the most recent start time.

Hints:
- Use `time.time()` to get the current Unix time (number of seconds since Jan 1, 1970)


In [16]:
import time

class CalledTooOftenError(Exception):
    pass

def once_per_minute(func):
    last_ran_at = 0
    def wrapper(*args):
        nonlocal last_ran_at
        current_time = time.time()

        if current_time - last_ran_at < 60:
            raise CalledTooOftenError('Too soon!')
        last_ran_at = current_time
        
        value = func(*args)
        return value
    return wrapper

@once_per_minute
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@once_per_minute
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))    
print(slow_mul(2, 3))
print(slow_add(2, 3))    
print(slow_mul(2, 3))

5
6


CalledTooOftenError: Too soon!

# Modifying inputs and outputs

When we write a decorator, we have two opportunities to do something:

1. When we define the function, and it's originally decorated. That is done by the outer function
2. When we run the function, meaning that the inner (`wrapper`) function is invoked.

This means that `wrapper` gets the arguments we wanted to pass to the original function, and it then invokes (or not) the original function.

It can use this opportunity to filter / modify the arguments that we pass, and also filter / modify any results that we return.

In [17]:
# Let's write a decorator that takes all of the arguments passed to
# a function, and assumes that they are all integers. It only passes
# along those integers that are odd.

def mysum(*numbers):
    total = 0

    for one_number in numbers:
        total += one_number

    return total

# normally, I could do this:
mysum(2,3,4,5)   

14

In [19]:
# but with my only_odds in place, we'll get 8 (i.e., 3 + 5)

def only_odds(func):
    def wrapper(*args):

        odd_numbers = []
        for one_item in args:
            if one_item % 2 == 1:
                odd_numbers.append(one_item)
                
        value = func(*odd_numbers)
        return value
    return wrapper

@only_odds
def mysum(*numbers):
    total = 0

    for one_number in numbers:
        total += one_number

    return total

# normally, I could do this:
mysum(2,3,4,5)   

8

In [ ]:
# let's tighten up the code a bit

def only_odds(func):
    def wrapper(*args):

        odd_numbers = []
        for one_item in args:
            if one_item % 2 == 1:
                odd_numbers.append(one_item)
                
        value = func(*odd_numbers)
        return value
    return wrapper

@only_odds
def mysum(*numbers):
    total = 0

    for one_number in numbers:
        total += one_number

    return total

# normally, I could do this:
mysum(2,3,4,5)   